In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt  
xlsx_file = "Empower.csv"
init_data = pd.read_csv(xlsx_file)
init_data

,SETTLEMENTDATE,REGIONID,TOTALDEMAND
0,2017-01-01 00:30:00.000,NSW1,7259.07000
1,2017-01-01 00:30:00.000,QLD1,6462.14000
2,2017-01-01 00:30:00.000,SA1,1395.21000
3,2017-01-01 00:30:00.000,TAS1,863.08000
4,2017-01-01 00:30:00.000,VIC1,4097.58000
...,...,...,...
481675,2022-07-01 00:00:00.000,NSW1,8731.47500
481676,2022-07-01 00:00:00.000,QLD1,5624.54500
481677,2022-07-01 00:00:00.000,SA1,1694.35000
481678,2022-07-01 00:00:00.000,TAS1,1134.75833


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
xlsx_file = "ElecMMS3_ROOFTOP_PV_ACTUAL_pivot.csv"
PV_pivot_data = pd.read_csv(xlsx_file)
# PV_pivot_data = PV_pivot_data.fillna(0)
print(str(PV_pivot_data['POWER_DAILY']))
print(type(init_data['SETTLEMENTDATE']))

init_data['added_PV_TOTALDEMAND'] = init_data['TOTALDEMAND']
for i in range(len(init_data)):
    if not pd.isna(PV_pivot_data['POWER_DAILY'][i]): 
        init_data['added_PV_TOTALDEMAND'][i] = PV_pivot_data['POWER_DAILY'][i] + init_data['TOTALDEMAND'][i]
    else:
        if pd.isna(PV_pivot_data['POWER_DAILY'][i]) and not pd.isna(PV_pivot_data['POWER_MEASUREMENT'][i]):
            init_data['added_PV_TOTALDEMAND'][i] = PV_pivot_data['POWER_MEASUREMENT'][i] + init_data['TOTALDEMAND'][i]
PV_pivot_data

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
481675    NaN
481676    NaN
481677    NaN
481678    NaN
481679    NaN
Name: POWER_DAILY, Length: 481680, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:
xlsx_file = "MarketData_dbo_Public_Holidays.csv"
Public_Holidays_data = pd.read_csv(xlsx_file)
Public_Holidays_data

In [ ]:
Public_Holidays_data['Timedate'] = pd.to_datetime(Public_Holidays_data['Date'], format="%Y-%m-%d")
Public_Holidays_data

In [ ]:
NSW_data = init_data[init_data.REGIONID == 'NSW1'].reset_index()
QLD_data = init_data[init_data.REGIONID == 'QLD1'].reset_index()
SA_data = init_data[init_data.REGIONID == 'SA1'].reset_index()
TAS_data = init_data[init_data.REGIONID == 'TAS1'].reset_index()
VIC_data = init_data[init_data.REGIONID == 'VIC1'].reset_index()
df = NSW_data
NSW_data

In [ ]:

def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
NSW_data['date'] = pd.to_datetime(df['SETTLEMENTDATE'], format="%Y-%m-%d %H:%M:%S.%f")

In [ ]:
%%time

Public_Holidays_data['date'] = pd.to_datetime(Public_Holidays_data['Date'], format="%Y-%m-%d %H:%M:%S.%f")
Public_Holidays_data
df['is_public_holiday'] = 0
for j in range(len(df)):
    for i in range(len(Public_Holidays_data)):
        if Public_Holidays_data['Official_NSW'][i] == True or Public_Holidays_data['AFMA_NSW'][i] == True or Public_Holidays_data['ASX_NSW'][i] == True:
            if Public_Holidays_data['Timedate'][i] == NSW_data['date'][j].date():
                df['is_public_holiday'][j] = 1

In [ ]:
data = pd.DataFrame(NSW_data.added_PV_TOTALDEMAND.copy())
data.columns = ["y"]

In [ ]:
# data.index = data.index.to_datetime()
data["hour"] = df.date.dt.hour
data["weekday"] = df.date.dt.weekday
data["month"] = df.date.dt.month
data["year"] = df.date.dt.year
data['is_weekend'] = data.weekday.isin([5,6])*1
data["is_public_holiday"] = df.is_public_holiday
data

In [ ]:
xlsx_file = "-33.86882_151.209295_Solcast_PT30M.csv"   #NSW
weather_data = pd.read_csv(xlsx_file)
weather_data['PeriodStart'][0]
weather_data['date'] = pd.to_datetime(weather_data['PeriodEnd'], format="%Y-%m-%dT%H:%M:%SZ")
weather_data
from datetime import timedelta
weather_data['date'] = weather_data['date'] + timedelta(hours=10)
weather_data
new_weather = weather_data.loc[weather_data['date']>='2017-01-01 01:00:00']
new_weather = new_weather.loc[new_weather['date']<='2022-07-01 00:00:00']
new_weather
new = pd.concat([new_weather,df],axis=0)
new_merge = df.merge(new_weather,on=['date'])
# # new_merge = new_merge.rename(columns={'TOTALDEMAND':'y'})
new_merge = new_merge.reset_index()
df_col = pd.concat([new_merge,data], axis=1)
train_data = df_col.drop(columns=['level_0','index','SETTLEMENTDATE','REGIONID','TOTALDEMAND','added_PV_TOTALDEMAND','date','PeriodEnd','PeriodStart','Period'])
train_data

In [ ]:


data = train_data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y = data.dropna().y
X = data.dropna().drop(['y'], axis=1)

X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# lognorm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from xgboost import XGBRegressor 
from xgboost_distribution import XGBDistribution

def plotModelResults(model, X_train=X_train, X_test=X_test, plot_intervals=False, plot_anomalies=False, scale=19.6):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """
    
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(600, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
#     plt.plot(y_test.values, label="actual", linewidth=2.0)

    print(y_test)
    print(prediction)
    plt.show()

xgb = XGBDistribution(distribution="log-normal")
xgb.fit(X_train_scaled, y_train)
testpred = xgb.predict(X_test_scaled)

mean, std = testpred.scale, testpred.s


plt.figure(figsize=(600, 7))
plt.plot(mean, "g", label="prediction", linewidth=2.0)
plt.plot(y_test.values, label="actual", linewidth=2.0)
plt.plot(mean+np.exp(std), "r--",label="upper", alpha=0.5,linewidth=2.0)
plt.plot(mean-np.exp(std),"r--", label="lower", alpha=0.5,linewidth=2.0)
error = mean_absolute_percentage_error(mean, y_test)
# print(y_test)
# print(prediction)
print(error)
plt.title("Mean absolute percentage error {0:.2f}%".format(error))
rmse = np.sqrt(mean_squared_error(mean, y_test))
print("RMSE: %f" % (rmse))
plt.legend(loc="best")
plt.tight_layout()
plt.grid(True);
plt.show()

print(std)


print(testpred)

In [ ]:
from scipy.stats import lognorm

fig = plt.figure(figsize =(180, 7))
ax = fig.add_axes([1, 1, 1, 1])
data = []
for i in range(200): #change to any numbers
    np.random.seed(1)

#generate log-normal distributed random variable with 1000 values
    lognorm_values = lognorm.rvs(s=testpred.s[i], scale=testpred.scale[i], size=1000)
    data.append(lognorm_values)
ax.set_xticklabels(X_test.hour[0:200])    
ax.boxplot(x=data)
# plt.plot(y_test_reset[0:200])
# ax.plot(y_test_reset[0:200],linewidth = 2.0)
# ax.plot(y_test[0:200])

plt.show()    

In [ ]:
plt.barh(X_train.columns, xgb.feature_importances_)

In [ ]:
X_train.columns

In [ ]:
xgb.feature_importances_

In [ ]:
# can change to specific number
AirTemp_weight = xgb.feature_importances_[1]
Azimuth_weight = xgb.feature_importances_[2]
CloudOpacity_weight = xgb.feature_importances_[3]
DewpointTemp_weight = xgb.feature_importances_[4]
Dhi_weight = xgb.feature_importances_[5]
Dni_weight = xgb.feature_importances_[6]
Ebh_weight = xgb.feature_importances_[7]
Ghi_weight = xgb.feature_importances_[8]
PrecipitableWater_weight = xgb.feature_importances_[9]
RelativeHumidity_weight = xgb.feature_importances_[10]
SnowWater_weight = xgb.feature_importances_[11]
SurfacePressure_weight = xgb.feature_importances_[12]
WindDirection10m_weight = xgb.feature_importances_[13]
WindSpeed10m_weight = xgb.feature_importances_[14]
Zenith_weight = xgb.feature_importances_[15]
AlbedoDaily_weight = xgb.feature_importances_[16]
hour_weight = xgb.feature_importances_[17]
weekday_weight = xgb.feature_importances_[18]
month_weight = xgb.feature_importances_[19]
year_weight = xgb.feature_importances_[20]
is_weekend_weight = xgb.feature_importances_[21]
is_public_holiday_weight = xgb.feature_importances_[22] 


In [ ]:
test = train_data.head(1)

In [ ]:
AirTemp_weight

In [ ]:
test['SurfacePressure'][0]

In [ ]:
min_diff = 9999
for index,row in df_col.iterrows():
    AirTemp = (row.AirTemp - float(test.AirTemp))**2 * AirTemp_weight
    Azimuth = (row.Azimuth - float(test.Azimuth))**2 * Azimuth_weight
    CloudOpacity = (row.CloudOpacity - float(test.CloudOpacity))**2 * CloudOpacity_weight
    DewpointTemp = (row.DewpointTemp - float(test.DewpointTemp))**2 * DewpointTemp_weight
    Dhi = (row.Dhi - float(test.Dhi))**2 * Dhi_weight
    Dni = (row.Dni - float(test.Dni))**2 * Dni_weight
    Ebh = (row.Ebh - float(test.Ebh))**2 * Ebh_weight
    Ghi = (row.Ghi - float(test.Ghi))**2 * Ghi_weight
    PrecipitableWater = (row.PrecipitableWater - float(test.PrecipitableWater))**2 * PrecipitableWater_weight
    RelativeHumidity = (row.RelativeHumidity - float(test.RelativeHumidity))**2 * RelativeHumidity_weight
    SnowWater = (row.SnowWater - float(test.SnowWater))**2 * SnowWater_weight
    SurfacePressure = (row.SurfacePressure - float(test.SurfacePressure))**2 * SurfacePressure_weight
    WindDirection10m = (row.WindDirection10m - float(test.WindDirection10m))**2 * WindDirection10m_weight
    WindSpeed10m = (row.WindSpeed10m - float(test.WindSpeed10m))**2 * WindSpeed10m_weight
    Zenith = (row.Zenith - float(test.Zenith))**2 * Zenith_weight
    AlbedoDaily = (row.AlbedoDaily - float(test.AlbedoDaily))**2 * AlbedoDaily_weight
    hour = (row.hour - float(test.hour))**2 * hour_weight
    weekday = (row.weekday - float(test.weekday))**2 * weekday_weight
    month = (row.month - float(test.month))**2 * month_weight
    year = (row.year - float(test.year))**2 * year_weight
    is_weekend = (row.is_weekend - float(test.is_weekend))**2 * is_weekend_weight
    is_public_holiday = (0 - 0)**2 * is_public_holiday_weight
    
    
    
    total_diff = AirTemp+Azimuth+CloudOpacity+DewpointTemp+Dhi+Dni+Ebh+Ghi+PrecipitableWater+RelativeHumidity+SnowWater+SurfacePressure+WindDirection10m+WindSpeed10m+Zenith+AlbedoDaily+hour+weekday+month+year+is_weekend+is_public_holiday   
    if total_diff < min_diff and row.year != 2017:
        min_diff = total_diff
        print(min_diff)
        print(row.date)
    

In [ ]:
float(test.AirTemp)